In [1]:
import pandas as pd
import time
from tqdm import tqdm
import re

In [2]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Define the file paths
lines_file = "/content/drive/MyDrive/Colab Notebooks/AAI-520/Final Project/GenAI-Chatbot/Cornell_Movie_Dialogue_Corpus/movie_lines.txt"
conversation_file = "/content/drive/MyDrive/Colab Notebooks/AAI-520/Final Project/GenAI-Chatbot/Cornell_Movie_Dialogue_Corpus/movie_conversations.txt"

# Read the lines file with specified encoding
lines = pd.read_csv(lines_file, sep=r'\s*\+\+\+\$\+\+\+\s*', header=None, engine='python', encoding='ISO-8859-1')
lines.columns = ['line_id', 'character_id', 'movie_id', 'character', 'text']

# Read the conversations file with specified encoding
conversations = pd.read_csv(conversation_file, sep=r'\s*\+\+\+\$\+\+\+\s*', header=None, engine='python', encoding='ISO-8859-1')
conversations.columns = ['character1_id', 'character2_id', 'movie_id', 'utterance_ids']

# Display the first few rows of each DataFrame
print(lines.head())
print(conversations.head())


  line_id character_id movie_id character          text
0   L1045           u0       m0    BIANCA  They do not!
1   L1044           u2       m0   CAMERON   They do to!
2    L985           u0       m0    BIANCA    I hope so.
3    L984           u2       m0   CAMERON     She okay?
4    L925           u0       m0    BIANCA     Let's go.
  character1_id character2_id movie_id                     utterance_ids
0            u0            u2       m0  ['L194', 'L195', 'L196', 'L197']
1            u0            u2       m0                  ['L198', 'L199']
2            u0            u2       m0  ['L200', 'L201', 'L202', 'L203']
3            u0            u2       m0          ['L204', 'L205', 'L206']
4            u0            u2       m0                  ['L207', 'L208']


In [4]:
# Function to clean the text
def clean_text(text):
    # Retain specific punctuation and remove others
    text = re.sub(r"[^\w\s.!?'’]", '', text)  # Keep punctuation and apostrophes
    text = text.strip()  # Strip whitespace
    text = ' '.join(text.split())  # Normalize whitespace
    return text

In [5]:
# Create a function to extract the conversation history and responses
def create_dataset(conversations, lines):
    # Create a dictionary for fast line access
    line_dict = dict(zip(lines['line_id'], lines['text']))

    dataset = []

    # Use tqdm to show progress
    for _, row in tqdm(conversations.iterrows(), total=conversations.shape[0], desc="Processing conversations"):
        utterance_ids = row['utterance_ids'].strip('[]').replace("'", "").split(', ')

        # Initialize conversation history
        conversation_history = []

        for utterance_id in utterance_ids:
            # Access the text from the dictionary
            text = line_dict.get(utterance_id)
            if isinstance(text, str):  # Ensure text is a string
                # Add the current line to the history
                conversation_history.append(text)

                # Create a record with the current history and response
                if len(conversation_history) > 1:
                    # The last entry is the response
                    response = conversation_history[-1]
                    history = conversation_history[:-1]

                    # Ensure that the history is made of strings
                    dataset.append({
                        'conversation_history': ' '.join(h for h in history if isinstance(h, str)),
                        'response': response
                    })

    return pd.DataFrame(dataset)




In [6]:
# Create the dataset
start_time = time.time()
chatbot_dataset = create_dataset(conversations, lines)
end_time = time.time()

# Clean the conversation history and responses, replacing the original columns
chatbot_dataset['conversation_history'] = chatbot_dataset['conversation_history'].apply(clean_text)
chatbot_dataset['response'] = chatbot_dataset['response'].apply(clean_text)

# Display the first few rows of the cleaned dataset
print(chatbot_dataset[['conversation_history', 'response']].head())
print(f"Execution Time: {end_time - start_time} seconds")



Processing conversations: 100%|██████████| 83097/83097 [00:04<00:00, 16708.68it/s]


                                conversation_history  \
0  Can we make this quick? Roxanne Korrine and An...   
1  Can we make this quick? Roxanne Korrine and An...   
2  Can we make this quick? Roxanne Korrine and An...   
3  You're asking me out. That's so cute. What's y...   
4  No no it's my fault we didn't have a proper in...   

                                            response  
0  Well I thought we'd start with pronunciation i...  
1  Not the hacking and gagging and spitting part....  
2  Okay... then how 'bout we try out some French ...  
3                                         Forget it.  
4                                           Cameron.  
Execution Time: 5.307232618331909 seconds


In [7]:
chatbot_dataset.head()

,conversation_history,response
0,Can we make this quick? Roxanne Korrine and An...,Well I thought we'd start with pronunciation i...
1,Can we make this quick? Roxanne Korrine and An...,Not the hacking and gagging and spitting part....
2,Can we make this quick? Roxanne Korrine and An...,Okay... then how 'bout we try out some French ...
3,You're asking me out. That's so cute. What's y...,Forget it.
4,No no it's my fault we didn't have a proper in...,Cameron.


In [8]:
chatbot_dataset.to_csv("/content/drive/MyDrive/Colab Notebooks/AAI-520/Final Project/GenAI-Chatbot/attempt_2.csv", index=False)

## Model


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.optim import AdamW
from tqdm import tqdm




In [ ]:
# Load your dataset (adjust the path as necessary)
# chatbot_dataset = pd.read_csv('path/to/your/cornell_movie_chatbot_dataset.csv')


In [10]:
# Split the dataset into train and test sets (80% train, 20% test)
train_data, test_data = train_test_split(chatbot_dataset, test_size=0.2, random_state=42)

# Further split the train data into training and validation sets (80% train, 20% validation)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

# Display the sizes of the datasets
print(f"Training set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")
print(f"Test set size: {len(test_data)}")



Training set size: 141674
Validation set size: 35419
Test set size: 44274


In [11]:
# Load the GPT-2 tokenizer with left padding
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', padding_side='left')
# Set pad token to be the same as the end-of-sequence token
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the datasets with time tracking
def tokenize_data(data):
    start_time = time.time()  # Start the timer
    # Use tqdm to show progress while processing the dataset
    encodings = tokenizer(
        list(tqdm(data['conversation_history'].tolist(), desc="Tokenizing")),  # Wrap the list with tqdm
        truncation=True,
        padding='max_length',  # Ensure all sequences are the same length
        max_length=100,  # Set maximum length for the sequences
        return_tensors='pt',  # Return PyTorch tensors
        return_attention_mask=True,  # Include the attention mask
        add_special_tokens=True,  # Add special tokens if necessary
    )
    end_time = time.time()  # End the timer
    print(f"Time taken for tokenization: {end_time - start_time:.2f} seconds")  # Print the elapsed time
    return encodings

# Tokenize each split
train_encodings = tokenize_data(train_data)
val_encodings = tokenize_data(val_data)
test_encodings = tokenize_data(test_data)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Time taken for tokenization: 42.80 seconds


Tokenizing: 100%|██████████| 35419/35419 [00:00<00:00, 2823009.53it/s]


Time taken for tokenization: 9.82 seconds


Tokenizing: 100%|██████████| 44274/44274 [00:00<00:00, 2611794.87it/s]


Time taken for tokenization: 12.49 seconds


In [12]:
class ChatbotDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings['input_ids'])

# Create datasets for training, validation, and testing
train_dataset = ChatbotDataset(train_encodings)
val_dataset = ChatbotDataset(val_encodings)
test_dataset = ChatbotDataset(test_encodings)


In [13]:
# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, num_workers=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, num_workers=4, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, num_workers=4, shuffle=False)


In [14]:
# Load the pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained('gpt2')

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [15]:
# Prepare input for response generation with context management and dynamic input length handling
def prepare_input(conversation_history, user_input, max_length=1024):
    # Combine conversation history and the latest user input
    full_input = ' '.join(conversation_history + [user_input])

    # Tokenize the combined input
    tokenized_input = tokenizer(full_input, truncation=False, return_tensors='pt')

    # If tokenized input exceeds max length, truncate the history
    while tokenized_input['input_ids'].size(1) > max_length:
        conversation_history.pop(0)  # Remove the oldest part of the conversation
        full_input = ' '.join(conversation_history + [user_input])
        tokenized_input = tokenizer(full_input, truncation=False, return_tensors='pt')

    return tokenized_input

In [16]:
# Step 9: Check for GPU Availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move the model to the GPU
model.to(device)

# Step 10: Define Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training Loop with Validation and Early Stopping
best_val_loss = float('inf')
patience = 3  # Number of epochs to wait for improvement
epochs_without_improvement = 0

# Step 11: Training Loop with Validation
for epoch in range(10):  # Number of epochs
    start_time = time.time()  # Start time for the epoch
    model.train()  # Set model to training mode

    # Training phase
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}"):
        optimizer.zero_grad()

        # Move input data to the GPU
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)


                # Compute the loss
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Validation step
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validating"):
            # Move input data to the GPU
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)  # Move attention mask to the device

            # Compute the loss
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
            val_loss += outputs.loss.item()

    # Calculate the time taken for the epoch
    end_time = time.time()
    epoch_time = end_time - start_time

    # Print training and validation losses along with epoch time
    print(f"Epoch: {epoch + 1}, Training Loss: {loss.item():.4f}, Validation Loss: {val_loss / len(val_loader):.4f}, Time: {epoch_time:.2f} seconds")

    # Check for improvement
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_without_improvement = 0  # Reset the counter
        # Optionally, save the model checkpoint here
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print(f"Early stopping triggered after {epoch + 1} epochs.")
            break


Using device: cuda


Validating: 100%|██████████| 2214/2214 [02:19<00:00, 15.82it/s]


Epoch: 1, Training Loss: 1.8100, Validation Loss: 1.1364, Time: 2135.54 seconds


Validating: 100%|██████████| 2214/2214 [02:19<00:00, 15.83it/s]


Epoch: 2, Training Loss: 1.1411, Validation Loss: 0.9782, Time: 2136.49 seconds


...

Epoch: 8, Training Loss: 0.5878, Validation Loss: 0.6063, Time: 2137.02 seconds


Validating: 100%|██████████| 2214/2214 [02:20<00:00, 15.81it/s]


Epoch: 9, Training Loss: 0.4761, Validation Loss: 0.5766, Time: 2137.07 seconds


Validating: 100%|██████████| 2214/2214 [02:20<00:00, 15.80it/s]

Epoch: 10, Training Loss: 0.3407, Validation Loss: 0.5573, Time: 2137.32 seconds


In [17]:
# Saving the model
gpt_model = "/content/drive/MyDrive/Colab Notebooks/AAI-520/Final Project/GenAI-Chatbot/eval/gpt/base_gpt_model_with_prep_input_10_epoch"
gpt_tokenizer = "/content/drive/MyDrive/Colab Notebooks/AAI-520/Final Project/GenAI-Chatbot/eval/gpt/base_gpt_tokenizer_with_prep_input_10_epoch"

# Save the model and tokenizer
model.save_pretrained(gpt_model)
tokenizer.save_pretrained(gpt_tokenizer)


('/content/drive/MyDrive/Colab Notebooks/AAI-520/Final Project/GenAI-Chatbot/eval/gpt/base_gpt_tokenizer_with_prep_input_10_epoch/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/AAI-520/Final Project/GenAI-Chatbot/eval/gpt/base_gpt_tokenizer_with_prep_input_10_epoch/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/AAI-520/Final Project/GenAI-Chatbot/eval/gpt/base_gpt_tokenizer_with_prep_input_10_epoch/vocab.json',
 '/content/drive/MyDrive/Colab Notebooks/AAI-520/Final Project/GenAI-Chatbot/eval/gpt/base_gpt_tokenizer_with_prep_input_10_epoch/merges.txt',
 '/content/drive/MyDrive/Colab Notebooks/AAI-520/Final Project/GenAI-Chatbot/eval/gpt/base_gpt_tokenizer_with_prep_input_10_epoch/added_tokens.json')

In [19]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=15f7e6e6bf66c90c1bf541852ebaa107ceb214deebdbc90aed6973b7e4284cff
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [20]:
import numpy as np
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score

In [21]:
# Sample function to compute the average loss and perplexity
def compute_average_loss_and_perplexity(loader, model, device):
    model.eval()
    total_loss = 0
    num_batches = 0

    # Start timing
    start_time = time.time()

    with torch.no_grad():
        # Use tqdm to show progress
        for batch in tqdm(loader, desc="Computing Average Loss and Perplexity"):
            input_ids = batch['input_ids'].to(device)
            outputs = model(input_ids=input_ids, labels=input_ids)
            loss = outputs.loss
            total_loss += loss.item()
            num_batches += 1

    avg_loss = total_loss / num_batches
    perplexity = np.exp(avg_loss)

    # Calculate the time taken
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time taken: {elapsed_time:.2f} seconds")

    return avg_loss, perplexity

# Assuming you have a validation DataLoader
avg_loss, perplexity = compute_average_loss_and_perplexity(val_loader, model, device)
print(f"Average Loss: {avg_loss:.4f}, Perplexity: {perplexity:.4f}")

Computing Average Loss and Perplexity: 100%|██████████| 2214/2214 [02:12<00:00, 16.65it/s]


Time taken: 133.00 seconds
Average Loss: 1.3344, Perplexity: 3.7976


In [22]:
# Function to compute BLEU score
def compute_bleu(predictions, references):
    # Calculate BLEU score for each prediction
    bleu_scores = [sentence_bleu([ref.split()], pred.split()) for pred, ref in zip(predictions, references)]
    avg_bleu = np.mean(bleu_scores)  # Average BLEU score across all predictions
    return avg_bleu


In [23]:
# Start the timer
start_time = time.time()

# Generate predictions
model.eval()
predictions = []
references = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Generating Predictions"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)  # Include the attention mask

        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=50,
            num_return_sequences=1
        )

        # Decode the generated output
        generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
        predictions.extend(generated_texts)

        # Collect reference texts
        reference_texts = [tokenizer.decode(ids, skip_special_tokens=True) for ids in input_ids]
        references.extend(reference_texts)

# End the timer
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken for generating predictions: {elapsed_time:.2f} seconds")

Generating Predictions: 100%|██████████| 2768/2768 [26:38<00:00,  1.73it/s]

Time taken for generating predictions: 1598.94 seconds


In [24]:
from nltk.translate.bleu_score import corpus_bleu
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

# Prepare the references in the required format for BLEU calculation
formatted_references = [[word_tokenize(ref)] for ref in references]  # Tokenize reference texts
formatted_predictions = [word_tokenize(pred) for pred in predictions]  # Tokenize generated texts

# Calculate BLEU score
bleu_score = corpus_bleu(formatted_references, formatted_predictions)
print(f"BLEU Score: {bleu_score:.4f}")



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


BLEU Score: 0.4049


In [25]:
pip install rouge-score


In [26]:
from rouge_score import rouge_scorer

# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Calculate ROUGE scores
rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

for ref, pred in zip(references, predictions):
    scores = scorer.score(ref, pred)
    rouge_scores['rouge1'].append(scores['rouge1'].fmeasure)
    rouge_scores['rouge2'].append(scores['rouge2'].fmeasure)
    rouge_scores['rougeL'].append(scores['rougeL'].fmeasure)

# Calculate average ROUGE scores
average_rouge1 = sum(rouge_scores['rouge1']) / len(rouge_scores['rouge1'])
average_rouge2 = sum(rouge_scores['rouge2']) / len(rouge_scores['rouge2'])
average_rougeL = sum(rouge_scores['rougeL']) / len(rouge_scores['rougeL'])

# Print average ROUGE scores
print(f"Average ROUGE-1 Score: {average_rouge1:.4f}")
print(f"Average ROUGE-2 Score: {average_rouge2:.4f}")
print(f"Average ROUGE-L Score: {average_rougeL:.4f}")


Average ROUGE-1 Score: 0.4776
Average ROUGE-2 Score: 0.4616
Average ROUGE-L Score: 0.4776


In [28]:
import nltk
nltk.download('wordnet')
from nltk.translate import meteor_score
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [30]:
# Calculate METEOR scores
meteor_scores = []

# Tokenize references and predictions
for ref, pred in zip(references, predictions):
    # Tokenize both reference and prediction
    tokenized_ref = word_tokenize(ref)
    tokenized_pred = word_tokenize(pred)

    # Calculate the METEOR score using tokenized inputs (as lists)
    score = meteor_score.single_meteor_score(tokenized_ref, tokenized_pred)
    meteor_scores.append(score)

# Calculate average METEOR score
average_meteor = sum(meteor_scores) / len(meteor_scores)

# Print average METEOR score
print(f"Average METEOR Score: {average_meteor:.4f}")



Average METEOR Score: 0.6768


In [18]:
# Load the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Set the pad token to the end-of-sequence token
tokenizer.pad_token = tokenizer.eos_token  # Assign pad token
tokenizer.pad_token_id = tokenizer.eos_token_id  # Ensure padding ID is the same as EOS

# Set the model to evaluation mode
model.eval()

print("Chatbot is ready! Type 'exit' to stop the conversation.")

while True:
    # Get user input
    input_text = input("You: ")

    # Check for exit condition
    if input_text.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break

    # Tokenize the input
    encoded_input = tokenizer.encode_plus(
        input_text,
        return_tensors='pt',
        padding='max_length',
        truncation=True,
        max_length=50,
        clean_up_tokenization_spaces=True  # Set to avoid warnings
    )

    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)

    # Generate output
    with torch.no_grad():
        output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=50,  # Generate up to 50 new tokens
            num_return_sequences=1
        )

    # Decode the generated output
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f"Chatbot: {response}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Chatbot is ready! Type 'exit' to stop the conversation.
You: hello


Keyword arguments {'clean_up_tokenization_spaces': True} not recognized.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: hello. Hello Leon. Hello Sonny. How are you doing? Well... I'm out of the hospital. Yeah. You said... I thought you were never getting out? I never thought I'd get out this way. I'll tell you. Well
You: what is your name


Keyword arguments {'clean_up_tokenization_spaces': True} not recognized.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: what is your nameby the way? Marylin?... Marylin?... Oh no. I just wanted to ask you something. And?...What kind of questions is that? To find out if you'll marry me. Yep. I don't think I'll
You: my name is aaron


Keyword arguments {'clean_up_tokenization_spaces': True} not recognized.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: my name is aaronof epic proportions. The aronof epic proportions? I'm not aetheanthic fairy. I'm a mythology actually. Well I suppose you could call me mythology? I'm not a mythology actually. I'm a kind of a science
You: your a myth?


Keyword arguments {'clean_up_tokenization_spaces': True} not recognized.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: your a myth? a myth... I'm not a myth John. I'm a doctor I'm a dermatologist I treat people with my teeth examined and I don't make a play for money. I treat people with broken bones. What are you talking about? I
You: exit
Chatbot: Goodbye!
